<h1 align="center"><font color="yellow">PGVector: Crie, armazene e consulte Embeddings OpenAI no PostgreSQL usando pgvector</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Aqui vamos aprender:

* Como usar o `PostgreSQL` como um banco de dados vetorial e armazenar dados de embeddings nele usando pgvector.

* Como usar Embeddings recuperadas (retrieved) de um banco de dados de vetores para aumentar a geração de LLM.


Usaremos o exemplo de criação de um chatbot para responder a perguntas sobre casos de uso do Timescale, fazendo referência ao conteúdo das postagens do blog Timescale Developer Q+A.

Este é um ótimo primeiro passo para criar algo como um chatbot que pode fazer referência a uma base de conhecimento da empresa ou a documentos do desenvolvedor.

Link de estudo:

* [pgvector](https://github.com/timescale/vector-cookbook/blob/main/openai_pgvector_helloworld/openai_pgvector_helloworld.ipynb)

* [LangChain and PGVector](https://github.com/timescale/vector-cookbook/blob/main/intro_langchain_pgvector/langchain_pgvector_intro.ipynb)

<font color="pink">Não esqueça de rodar a Imagem de `pgvector`:</font>

* docker pull ankane/pgvector

* docker-compose up -d 

In [2]:
%pip show langchain

Name: langchain
Version: 0.0.184
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/venv_LLMs/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip show openai

Name: openai
Version: 0.27.6
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/venv_LLMs/lib/python3.10/site-packages
Requires: aiohttp, requests, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


# Parte 1: <font color="red">Criamos os Embeddings</font>

In [42]:
import openai
import os
import pandas as pd
import numpy as np
import json
import tiktoken
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector


# Load your CSV file into a pandas DataFrame
df = pd.read_csv('blog_posts_data.csv')
df.head()

,title,content,url
0,"How to Build a Weather Station With Elixir, Ne...",This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-to-build-a-...
1,CloudQuery on Using PostgreSQL for Cloud Asset...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/cloudquery-on-u...
2,How a Data Scientist Is Building a Time-Series...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-a-data-scie...
3,How Conserv Safeguards History: Building an En...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-conserv-saf...
4,How Messari Uses Data to Open the Cryptoeconom...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-messari-use...


In [43]:
df.shape

(24, 3)

In [44]:
print(df['title'].iloc[0])
print("")
print(df['content'].iloc[0])
print("")
print(df['url'].iloc[0])

How to Build a Weather Station With Elixir, Nerves, and TimescaleDB

This is an installment of our “Community Member Spotlight” series, where we invite our customers to share their work, shining a light on their success and inspiring others with new ways to use technology to solve problems.In this edition,Alexander Koutmos, author of the Build a Weather Station with Elixir and Nerves book, joins us to share how he uses Grafana and TimescaleDB to store and visualize weather data collected from IoT sensors.About the teamThe bookBuild a Weather Station with Elixir and Nerveswas a joint effort between Bruce Tate, Frank Hunleth, and me.I have been writing software professionally for almost a decade and have been working primarily with Elixir since 2016. I currently maintain a few Elixir libraries onHexand also runStagira, a software consultancy company.Bruce Tateis a kayaker, programmer, and father of two from Chattanooga, Tennessee. He is the author of more than ten books and has been arou

### 1.1 Calculamos o custo de Embeddings de nosso Dataset

<font color="orange">Geralmente, é uma boa ideia calcular quanto custará a criação de Embeddings para o conteúdo selecionado. Usamos várias funções auxiliares para calcular uma estimativa de custo antes de criar as incorporações para nos ajudar a evitar surpresas.

Para este exemplo de brinquedo, como estamos usando um pequeno conjunto de dados, o custo total será inferior a `US$ 0,01.`</font>

In [45]:
# Helper functions to help us create the embeddings

# Helper func: calculate number of tokens
def num_tokens_from_string(string: str, encoding_name = "cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Helper function: calculate length of essay
def get_essay_length(essay):
    word_list = essay.split()
    num_words = len(word_list)
    return num_words

# Helper function: calculate cost of embedding num_tokens
# Assumes we're using the text-embedding-ada-002 model
# See https://openai.com/pricing
def get_embedding_cost(num_tokens):
    return num_tokens/1000*0.0001

# Helper function: calculate total cost of embedding all content in the dataframe
def get_total_embeddings_cost():
    total_tokens = 0
    for i in range(len(df.index)):
        text = df['content'][i]
        token_len = num_tokens_from_string(text)
        total_tokens = total_tokens + token_len
    total_cost = get_embedding_cost(total_tokens)
    return total_cost

# Helper function: get embeddings for a text
def get_embeddings(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input = text.replace("\n"," ")
    )
    embedding = response['data'][0]['embedding']
    return embedding


In [46]:
# verificação rápida do valor total do token para estimativa de preço
total_cost = get_total_embeddings_cost()
print("Preço estimado para Embeddings deste conteúdo = $" + str(total_cost))

Preço estimado para Embeddings deste conteúdo = $0.0060178


### 1.2 Criamos chunks menores de conteúdo

A API OpenAI tem um limite para a quantidade máxima de `tokens` que ela cria para criar um Embedding em uma única solicitação. Para contornar esse limite, vamos dividir nosso texto em partes menores. Em geral, é uma prática recomendada criar embeddings de um determinado tamanho para obter uma melhor recuperação. Para nossos propósitos, buscaremos pedaços de cerca de `512` tokens cada.

<font color="red">NOTA:</font>

se preferir pular esta etapa, você pode usar o arquivo fornecido: `blog_data_and_embeddings.csv`, que contém os dados e Embeddings que você gerará nesta etapa.

In [47]:
###############################################################################
# Crie uma nova lista com pequenos chunks de conteúdo para não atingir os limites máximos de token
# Note: o número máximo de tokens para uma única solicitação é 8191
# https://openai.com/docs/api-reference/requests
###############################################################################
# list for chunked content and embeddings
new_list = []
# Divida o texto em tamanhos de token de cerca de 512 tokens
for i in range(len(df.index)):
    text = df['content'][i]
    token_len = num_tokens_from_string(text)
    if token_len <= 512:
        new_list.append([df['title'][i], df['content'][i], df['url'][i], token_len])
    else:
        # add content to the new list in chunks
        start = 0
        ideal_token_size = 512
        # 1 token ~ 3/4 of a word
        ideal_size = int(ideal_token_size // (4/3))
        end = ideal_size
        #split text by spaces into words
        words = text.split()

        #remove empty spaces
        words = [x for x in words if x != ' ']

        total_words = len(words)
        
        #calculate iterations
        chunks = total_words // ideal_size
        if total_words % ideal_size != 0:
            chunks += 1
        
        new_content = []
        for j in range(chunks):
            if end > total_words:
                end = total_words
            new_content = words[start:end]
            new_content_string = ' '.join(new_content)
            new_content_token_len = num_tokens_from_string(new_content_string)
            if new_content_token_len > 0:
                new_list.append([df['title'][i], new_content_string, df['url'][i], new_content_token_len])
            start += ideal_size
            end += ideal_size

In [48]:
# Crie Embeddings para cada parte do conteúdo:
for i in range(len(new_list)):
    text = new_list[i][1]
    embedding = get_embeddings(text)
    new_list[i].append(embedding)

# Create a new dataframe from the list
df_new = pd.DataFrame(new_list, columns=['title', 'content', 'url', 'tokens', 'embeddings'])
df_new.head()

,title,content,url,tokens,embeddings
0,"How to Build a Weather Station With Elixir, Ne...",This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-to-build-a-...,501,"[0.021440856158733368, 0.02200360782444477, -0..."
1,"How to Build a Weather Station With Elixir, Ne...",capture weather and environmental data. In all...,https://www.timescale.com/blog/how-to-build-a-...,512,"[0.01620873250067234, 0.011362895369529724, 0...."
2,"How to Build a Weather Station With Elixir, Ne...",command in their database migration:SELECT cre...,https://www.timescale.com/blog/how-to-build-a-...,374,"[0.022517921403050423, -0.0019158280920237303,..."
3,CloudQuery on Using PostgreSQL for Cloud Asset...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/cloudquery-on-u...,519,"[0.008915113285183907, -0.004873732570558786, ..."
4,CloudQuery on Using PostgreSQL for Cloud Asset...,Architecture with CloudQuery SDK- Writing plug...,https://www.timescale.com/blog/cloudquery-on-u...,511,"[0.0204352755099535, 0.010087345726788044, 0.0..."


In [49]:
df_new.shape

(129, 5)

In [50]:
# Salve o dataframe com incorporações como um arquivo CSV
df_new.to_csv('blog_data_and_embeddings.csv', index=False)

# Também pode ser útil salvar como um arquivo JSON, mas não usaremos isso no tutorial
#df_new.to_json('blog_data_and_embeddings.json')


# Parte 2: <font color="red">Armazenar Embeddings com pgvector</font>

Nesta seção, armazenaremos nossos Embeddings e metadados associados.

Usaremos o `PostgreSQL` como banco de dados vetorial, com a extensão `pgvector`.

Você pode criar um banco de dados PostgreSQL em nuvem gratuitamente no Timescale ou usar um banco de dados PostgreSQL local para esta etapa.

### 2.1 Conecte-se e configure seu vector Database

In [60]:
# Timescale database connection string
# Found under "Service URL" of the credential cheat-sheet or "Connection Info" in the Timescale console
# In terminal, run: export TIMESCALE_CONNECTION_STRING=postgres://<fill in here>

import psycopg2
from langchain.vectorstores.pgvector import PGVector

# Constrói a string de conexão PGVector a partir dos parâmetros.
host= os.environ['DB_HOST']
port= os.environ['DB_PORT']
user= os.environ['DB_USER']
password= os.environ['DB_PASSWORD']
dbname= os.environ['DB_NAME']

# Usamos postgresql em vez de postgres na string conn, já que LangChain usa sqlalchemy sob o capô
# Você pode remover o ?sslmode=require se tiver uma instância local do PostgreSQL rodando sem SSL

#CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}?sslmode=require"
CONNECTION_STRING = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
#connection_string  = os.environ['TIMESCALE_CONNECTION_STRING'] 

# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(CONNECTION_STRING)
cur = conn.cursor()

# Install pgvector 
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()

# Register the vector type with psycopg2
register_vector(conn)

# Criar tabela para armazenar embeddings e metadados (metadata):
table_create_command = """
CREATE TABLE IF NOT EXISTS Tabela_Eddy_Embeddings (
            id bigserial primary key, 
            title text,
            url text,
            content text,
            tokens integer,
            embedding vector(1536)
            );
            """

cur.execute(table_create_command)
cur.close()
conn.commit()


<font color="orange">Opcional: remova o comentário e execute o código a seguir apenas se precisar ler as incorporações e os metadados do arquivo CSV fornecido</font>

In [52]:
# Descomente e execute esta célula apenas se precisar ler os dados do blog e incorporações do arquivo CSV fornecido
# Caso contrário, pule para a próxima célula
'''
df = pd.read_csv('blog_data_and_embeddings.csv')
titles = df['title']
urls = df['url']
contents = df['content']
tokens = df['tokens']
embeds = [list(map(float, ast.literal_eval(embed_str))) for embed_str in df['embeddings']]

df_new = pd.DataFrame({
    'title': titles,
    'url': urls,
    'content': contents,
    'tokens': tokens,
    'embeddings': embeds
})
'''

"\ndf = pd.read_csv('blog_data_and_embeddings.csv')\ntitles = df['title']\nurls = df['url']\ncontents = df['content']\ntokens = df['tokens']\nembeds = [list(map(float, ast.literal_eval(embed_str))) for embed_str in df['embeddings']]\n\ndf_new = pd.DataFrame({\n    'title': titles,\n    'url': urls,\n    'content': contents,\n    'tokens': tokens,\n    'embeddings': embeds\n})\n"

### 2.2 Ingerir e armazenar dados vetoriais no `PostgreSQL` usando `pgvector`

Nesta seção, inseriremos em batch nossos Embeddings e metadados no `PostgreSQL` e também criaremos um índice para ajudar a acelerar a pesquisa.

In [61]:
register_vector(conn)
cur = conn.cursor()


In [62]:
# Lembre-se da estrutura do dataframe
df_new.head()

,title,content,url,tokens,embeddings
0,"How to Build a Weather Station With Elixir, Ne...",This is an installment of our “Community Membe...,https://www.timescale.com/blog/how-to-build-a-...,501,"[0.021440856158733368, 0.02200360782444477, -0..."
1,"How to Build a Weather Station With Elixir, Ne...",capture weather and environmental data. In all...,https://www.timescale.com/blog/how-to-build-a-...,512,"[0.01620873250067234, 0.011362895369529724, 0...."
2,"How to Build a Weather Station With Elixir, Ne...",command in their database migration:SELECT cre...,https://www.timescale.com/blog/how-to-build-a-...,374,"[0.022517921403050423, -0.0019158280920237303,..."
3,CloudQuery on Using PostgreSQL for Cloud Asset...,This is an installment of our “Community Membe...,https://www.timescale.com/blog/cloudquery-on-u...,519,"[0.008915113285183907, -0.004873732570558786, ..."
4,CloudQuery on Using PostgreSQL for Cloud Asset...,Architecture with CloudQuery SDK- Writing plug...,https://www.timescale.com/blog/cloudquery-on-u...,511,"[0.0204352755099535, 0.010087345726788044, 0.0..."


<font color="orange">Embeddings de inserção em lote usando `execute_values()` do `psycopg2`.</font>

In [63]:
# Lote insere Embeddings e metadados do dataframe no banco de dados PostgreSQL

# Prepare the list of tuples to insert:
data_list = [(row['title'], row['url'], row['content'], int(row['tokens']), np.array(row['embeddings'])) for index, row in df_new.iterrows()]

# Use execute_values to perform batch insertion:
execute_values(cur, "INSERT INTO Tabela_Eddy_Embeddings (title, url, content, tokens, embedding) VALUES %s", data_list)

# Commit after we insert all embeddings
conn.commit()


<font color="orange">Verificação de sanidade executando algumas consultas simples na tabela de Embeddings.</font>

In [64]:
cur.execute("SELECT COUNT(*) as cnt FROM Tabela_Eddy_Embeddings;")

num_records = cur.fetchone()[0]
print("Número de registros vetoriais na tabela: ", num_records,"\n")
# A saída correta deve ser 129

Número de registros vetoriais na tabela:  129 



In [65]:
# imprima o primeiro registro na tabela, para verificação de sanidade
cur.execute("SELECT * FROM Tabela_Eddy_Embeddings LIMIT 1;")

records = cur.fetchall()
print("Primeiro registro na tabela: ", records)


Primeiro registro na tabela:  [(1, 'How to Build a Weather Station With Elixir, Nerves, and TimescaleDB', 'https://www.timescale.com/blog/how-to-build-a-weather-station-with-elixir-nerves-and-timescaledb/', 'This is an installment of our “Community Member Spotlight” series, where we invite our customers to share their work, shining a light on their success and inspiring others with new ways to use technology to solve problems.In this edition,Alexander Koutmos, author of the Build a Weather Station with Elixir and Nerves book, joins us to share how he uses Grafana and TimescaleDB to store and visualize weather data collected from IoT sensors.About the teamThe bookBuild a Weather Station with Elixir and Nerveswas a joint effort between Bruce Tate, Frank Hunleth, and me.I have been writing software professionally for almost a decade and have been working primarily with Elixir since 2016. I currently maintain a few Elixir libraries onHexand also runStagira, a software consultancy company.B

<font color="yellow">Crie um índice na coluna de `embedding` para uma comparação de `similaridade de cosseno` mais rápida.</font>

In [66]:
# Crie um índice nos dados para uma recuperação (retrieval) mais rápida. 
# Isso não é realmente necessário para 129 vetores, mas mostra o uso para conjuntos de dados maiores.

# OBSERVAÇÃO: ---> sempre crie esse tipo de índice depois de ter os dados já inseridos no BD

# Calcular os parâmetros do índice de acordo com as melhores práticas:
num_lists = num_records / 1000
if num_lists < 10:
    num_lists = 10
if num_records > 1000000:
    num_lists = math.sqrt(num_records)

# Use a medida de distância do cosseno, que é o que usaremos mais tarde para queries:
cur.execute(f'CREATE INDEX ON Tabela_Eddy_Embeddings USING ivfflat (embedding vector_cosine_ops) WITH (lists = {num_lists});')
conn.commit() 

# Parte 3: <font color="red">Pesquisa de vizinho mais próximo usando pgvector - `Nearest Neighbor Search using pgvector`</font>

Nesta parte final do tutorial, vamos fazer queries em nossa `Tabela_Eddy_Embeddings`.

Mostraremos um exemplo de `RAG`: `Retrieval Augmented Generation`, onde recuperaremos dados relevantes de nosso banco de dados de vetores e os daremos ao `LLM` como contexto para usar quando ele gerar uma resposta a um prompt.

In [91]:
# Função auxiliar: obteha completion de texto da API da OpenAI
# Observe que o máximo de tokens é 4097
# Observe que estamos usando o modelo gpt-3.5-turbo-0613 mais recente

def get_completion_from_messages(messages, model="gpt-3.5-turbo-0613", temperature=0, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]


In [92]:
# Função auxiliar: obtenha os 3 documentos mais SIMILARES do banco de dados:

def get_top3_similar_docs(query_embedding, conn):
    embedding_array = np.array(query_embedding)

    # Register pgvector extension
    register_vector(conn)
    cur = conn.cursor()

    # Obtemos os 3 principais documentos mais similares usando o OPERADOR <=> KNN
    cur.execute("SELECT content FROM Tabela_Eddy_Embeddings ORDER BY embedding <=> %s LIMIT 3", (embedding_array,))
    top3_docs = cur.fetchall()
    return top3_docs


### 3.1 Definir um prompt para o LLM

Aqui definiremos o `prompt` para o qual queremos que o `LLM` forneça uma resposta.

Escolhemos um exemplo relevante para os dados de postagem do blog armazenados no banco de dados.

In [93]:
# Pergunta sobre escala de tempo que queremos que o modelo responda:
#input = "How is Timescale used in IoT?"
input = "Como a escala de tempo é usada na IoT?"


In [99]:
# Função para processar a entrada com recuperação dos documentos mais similares do banco de dados:
def process_input_with_retrieval(user_input):
    delimiter = "```"

    # Step 1: Obtenha documentos relacionados à entrada do usuário do banco de dados
    related_docs = get_top3_similar_docs(get_embeddings(user_input), conn)

    # Step 2: Obtenha a COMPLETION da API OpenAI:
    # Defina a mensagem do sistema para ajudar a definir o tom e o contexto apropriados para o modelo
    system_message = f"""
    Você é um chatbot amigável. \
    Você pode responder a perguntas, apenas em português, sobre o timescaledb, seus recursos e casos de uso. \
    Você responde em um tom conciso e tecnicamente confiável. \
    """

    # Preparar mensagens para passar ao modelo.
    # Usamos um delimitador para ajudar o modelo a entender onde o user_input começa e termina:
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"{delimiter}{user_input}{delimiter}"},
        {"role": "assistant", "content": f"Informações relevantes dos estudos de caso da escala de tempo (Timescale): \n\n {related_docs[0][0]} \n\n {related_docs[1][0]} \n\n {related_docs[2][0]}"}   
    ]
    #print(messages)
    final_response = get_completion_from_messages(messages)
    return final_response


In [100]:
response = process_input_with_retrieval(input)
print(input)
print("")
print(response)


[{'role': 'system', 'content': '\n    Você é um chatbot amigável.     Você pode responder a perguntas, apenas em português, sobre o timescaledb, seus recursos e casos de uso.     Você responde em um tom conciso e tecnicamente confiável.     '}, {'role': 'user', 'content': '```Como a escala de tempo é usada na IoT?```'}, {'role': 'assistant', 'content': "Informações relevantes dos estudos de caso da escala de tempo (Timescale): \n\n 📬This blog post was originally published in February 2021 and updated in February 2023 to reflect Everactive's data stack and business evolution.This is an installment of our “Community Member Spotlight” series, where we invite our customers to share their work, shining a light on their success and inspiring others with new ways to use technology to solve problems.In this edition,Carlos Olmos,Dan Wright, andClayton Yochumfrom Everactive join us to share how they’re bringing analytics and real-time device monitoring to scenarios and places never before possib

In [96]:
# Também podemos fazer perguntas ao modelo sobre documentos específicos no banco de dados

#input_2 = "Tell me about Edeva and Hopara. How do they use Timescale?"
input_2 = "Conte-me sobre Edeva e Hopara. Como eles usam a escala de tempo?"
response_2 = process_input_with_retrieval(input_2)
print(input_2)
print(response_2)


Conte-me sobre Edeva e Hopara. Como eles usam a escala de tempo?
Edeva e Hopara são empresas que utilizam o TimescaleDB para armazenar e consultar dados de séries temporais em tempo real. Eles usam a escala de tempo para monitorar e visualizar dados de sensores em aplicações de monitoramento em tempo real, como monitoramento de máquinas e ativos. O TimescaleDB permite que eles armazenem grandes volumes de dados de séries temporais e executem consultas eficientes para análise e visualização em tempo real.
